# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,betio village,1.3580,172.9211,29.02,81,100,7.42,KI,1733864941
1,1,hamilton,39.1834,-84.5333,9.92,94,100,4.47,US,1733864896
2,2,new norfolk,-42.7826,147.0587,12.73,69,99,3.32,AU,1733864944
3,3,adamstown,-25.0660,-130.1015,23.14,78,48,5.98,PN,1733864684
4,4,waitangi,-43.9535,-176.5597,16.01,84,77,0.89,NZ,1733864946


In [4]:
# Date Cleaning w/dates
city_data_df["Date"] = pd.to_datetime(city_data_df.Date * 1e9)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,betio village,1.3580,172.9211,29.02,81,100,7.42,KI,2024-12-10 21:09:01
1,1,hamilton,39.1834,-84.5333,9.92,94,100,4.47,US,2024-12-10 21:08:16
2,2,new norfolk,-42.7826,147.0587,12.73,69,99,3.32,AU,2024-12-10 21:09:04
3,3,adamstown,-25.0660,-130.1015,23.14,78,48,5.98,PN,2024-12-10 21:04:44
4,4,waitangi,-43.9535,-176.5597,16.01,84,77,0.89,NZ,2024-12-10 21:09:06


In [6]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     576 non-null    int64         
 1   City        576 non-null    object        
 2   Lat         576 non-null    float64       
 3   Lng         576 non-null    float64       
 4   Max Temp    576 non-null    float64       
 5   Humidity    576 non-null    int64         
 6   Cloudiness  576 non-null    int64         
 7   Wind Speed  576 non-null    float64       
 8   Country     571 non-null    object        
 9   Date        576 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 45.1+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values

narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 10.0)]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,adamstown,-25.0660,-130.1015,23.14,78,48,5.98,PN,2024-12-10 21:04:44
8,8,porto novo,6.4965,2.6036,26.34,75,4,3.03,BJ,2024-12-10 21:09:10
9,9,bengkulu,-3.8004,102.2655,23.76,88,73,3.18,ID,2024-12-10 21:06:23
10,10,avarua,-21.2078,-159.7750,26.03,83,75,9.26,CK,2024-12-10 21:09:12
20,20,urangan,-25.3000,152.9000,23.22,84,100,5.39,AU,2024-12-10 21:09:23
...,...,...,...,...,...,...,...,...,...,...
555,555,saint-pierre,-21.3393,55.4781,24.82,78,0,6.69,RE,2024-12-10 21:19:46
559,559,lekki,6.3607,4.8333,24.23,80,98,1.04,NG,2024-12-10 21:19:52
565,565,sadjoavato,-12.6334,49.3490,23.17,81,60,2.41,MG,2024-12-10 21:19:58
569,569,irara,-12.0500,-38.7667,26.86,49,3,7.02,BR,2024-12-10 21:20:03


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,adamstown,PN,-25.0660,-130.1015,78,
8,porto novo,BJ,6.4965,2.6036,75,
9,bengkulu,ID,-3.8004,102.2655,88,
10,avarua,CK,-21.2078,-159.7750,83,
20,urangan,AU,-25.3000,152.9000,84,
...,...,...,...,...,...,...
555,saint-pierre,RE,-21.3393,55.4781,78,
559,lekki,NG,6.3607,4.8333,80,
565,sadjoavato,MG,-12.6334,49.3490,81,
569,irara,BR,-12.0500,-38.7667,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

for index, row in hotel_df.iterrows():
    # from the Weather CSV  
    latitude = row["Lat"]
    longitude = row["Lng"]

    
    # Set the parameters for the type of search
    params['filter'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"
   
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)
    data = response.json()
    # Run request
    try:
      hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

adamstown - nearest hotel: No hotel found
porto novo - nearest hotel: Hôtel 6500 CFA
bengkulu - nearest hotel: Dena Hotel
avarua - nearest hotel: Paradise Inn
urangan - nearest hotel: Ramada
greenville - nearest hotel: Quality Inn
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
kirakira - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
pahokee - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
san mariano - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
butterworth - nearest hotel: No hotel found
mukalla - nearest hotel: فندق الجابري
dongola - nearest hotel: Lord
arraial do cabo - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
tanjung selor - nearest hotel: No hotel found
ebolowa - nearest hotel: Hôtel le Relais
west island - nearest hotel: Cocos Village Bungalows
jayapura - nearest hotel: Hotel Jasmine Jayapura
emerald - nearest hotel: Route 66 Motor Inn
sao felix do

,City,Country,Lat,Lng,Humidity,Hotel Name
3,adamstown,PN,-25.0660,-130.1015,78,No hotel found
8,porto novo,BJ,6.4965,2.6036,75,Hôtel 6500 CFA
9,bengkulu,ID,-3.8004,102.2655,88,Dena Hotel
10,avarua,CK,-21.2078,-159.7750,83,Paradise Inn
20,urangan,AU,-25.3000,152.9000,84,Ramada
...,...,...,...,...,...,...
555,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
559,lekki,NG,6.3607,4.8333,80,No hotel found
565,sadjoavato,MG,-12.6334,49.3490,81,No hotel found
569,irara,BR,-12.0500,-38.7667,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Hotel Name"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)